In [3]:
from google.colab import drive
drive.mount ("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [97]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout, Flatten, TimeDistributed, LSTM, Input, BatchNormalization, Conv2D, MaxPooling2D, Reshape, Conv1D, GlobalAveragePooling1D, MaxPooling1D, Lambda
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy
from keras.losses import categorical_crossentropy
import tensorflow_hub as hub
from PIL import Image
import gzip
from keras.preprocessing.sequence import pad_sequences
import spacy
import h5py
import os
import cv2
import pickle
import re
import shutil
import glob
import gzip
%matplotlib inline

In [98]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from sklearn.model_selection import train_test_split
import pandas as pd
import unicodedata
import re
import numpy as np
import os
import io
import time
from string import digits

In [99]:
path_gloss_files = '/content/drive/MyDrive/Colab Notebooks/phoenix14t.pami0.train.annotations_only.gzip'
path_dev_gloss = '/content/drive/MyDrive/Colab Notebooks/phoenix14t.pami0.dev.annotations_only.gzip'
path_test_gloss = '/content/drive/MyDrive/Colab Notebooks/phoenix14t.pami0.test.annotations_only.gzip'

# reading glooss files

In [100]:
with gzip.open(path_gloss_files,'rb') as f:
  annotation_gloss = pickle.load(f)

In [101]:
with gzip.open(path_dev_gloss,'rb') as f:
  annotation_dev = pickle.load(f)

In [102]:
with gzip.open(path_test_gloss,'rb')as f :
  annotation_test = pickle.load(f)

In [103]:
annotation_gloss = pd.DataFrame(annotation_gloss)

In [104]:
annotation_gloss_dev = pd.DataFrame(annotation_dev)



In [105]:
annotation_gloss_test = pd.DataFrame(annotation_test)

# preprocessing gloes

In [106]:
for i in range(len(annotation_gloss)):
  annotation_gloss.iloc[i,2] = 'startseq '+annotation_gloss.iloc[i,2].lower() + ' endseq'
  annotation_gloss.iloc[i,3] = 'startseq '+annotation_gloss.iloc[i,3].lower() + ' endseq'
  annotation_gloss.iloc[i,3] = annotation_gloss.iloc[i,3].replace(' .','')
  #annotation_gloss.iloc[i,3] =annotation_gloss.iloc[i,3][-2].replace(' ','')
 

In [107]:
for i in range(len(annotation_gloss_dev)):
  annotation_gloss_dev.iloc[i,2] = 'startseq '+annotation_gloss_dev.iloc[i,2].lower() + ' endseq'
  annotation_gloss_dev.iloc[i,3] = 'startseq '+annotation_gloss_dev.iloc[i,3].lower() + ' endseq'
  annotation_gloss_dev.iloc[i,3] = annotation_gloss_dev.iloc[i,3].replace(' .','')

In [108]:
for i in range(len(annotation_gloss_test)):
  annotation_gloss_test.iloc[i,2] = 'startseq '+annotation_gloss_test.iloc[i,2].lower() + ' endseq'
  annotation_gloss_test.iloc[i,3] = 'startseq '+annotation_gloss_test.iloc[i,3].lower() + ' endseq'
  annotation_gloss_test.iloc[i,3] = annotation_gloss_test.iloc[i,3].replace(' .','')

In [109]:
annotation_gloss.iloc[1,2]

'startseq ort regen durch regen koennen ueberschwemmung koennen endseq'

In [110]:
annotation_gloss.iloc[1,3]

'startseq mancherorts regnet es auch länger und ergiebig auch lokale überschwemmungen sind wieder möglich endseq'

# calculating max length

In [176]:
gloss_gloss = []
for i in range(7096):
  gloss_gloss.append(annotation_gloss.iloc[i,3])
  gloss_gloss.append(annotation_gloss.iloc[i,2])
for i in range(len(annotation_gloss_dev)):
  gloss_gloss.append(annotation_gloss_dev.iloc[i,3])
  gloss_gloss.append(annotation_gloss_dev.iloc[i,2])
for i in range(len(annotation_gloss_test)):
  gloss_gloss.append(annotation_gloss_test.iloc[i,3])
  gloss_gloss.append(annotation_gloss_test.iloc[i,2])

In [177]:
len(gloss_gloss)

16514

In [178]:
length_check = []
for i in range (len(gloss_gloss)):
  length_check.append(gloss_gloss[i].split(' '))
  


In [179]:
count=0
for i in range(len(length_check)):
  if(len(length_check[i])>count):
    count = len(length_check[i])

In [180]:
count

54

In [173]:
length_check[0]

['startseq',
 'und',
 'nun',
 'die',
 'wettervorhersage',
 'für',
 'morgen',
 'donnerstag',
 'den',
 'zwölften',
 'august',
 'endseq']

In [174]:
for i in range(len(length_check)):
  for j in length_check[i]:
    if(j=='region'):
      print("hello")
 

# creating unique words list

In [181]:
unique_list_text = []
for i in range(len(length_check)):
  for j in length_check[i]:
    if (j not in unique_list_text):
      unique_list_text.append(j)

In [188]:
len(unique_list_text)

3606

# removign less frequency words

In [183]:
dictionary_unq = {}


In [26]:
for i in range(len(unique_list_text)):
  count= 0
  for j in length_check:
    for k in j:
      if(unique_list_text[i] == k):
        count+=1
  dictionary_unq[unique_list_text[i]] = count


In [27]:
for keys in dictionary_unq.keys():
  if dictionary_unq[keys]<5:
    unique_list_text.remove(keys)

In [28]:
unique_list_text.append('<UNK>')

In [118]:
len(unique_list_text)

3002

# creating tokenizer

In [189]:
def create_tokenizer(unique_list_text):
  tokenizer = tf.keras.preprocessing.text.Tokenizer()
  tokenizer.fit_on_texts(unique_list_text)
  return tokenizer


In [190]:
tokenizer = create_tokenizer(unique_list_text)

In [191]:
vocab_size_text = len(tokenizer.word_index)+1

In [192]:
print("vocalb size",vocab_size_text)

vocalb size 3434


# creating dataset for tokenizeer


In [193]:
gloss_token = []
text_token = []
for i in range(len(annotation_gloss)):
  gloss_token.append(annotation_gloss.iloc[i,2])
  text_token.append(annotation_gloss.iloc[i,3])

In [194]:
gloss_token[0]

'startseq jetzt wetter morgen donnerstag zwoelf februar endseq'

In [195]:
text_token[0]

'startseq und nun die wettervorhersage für morgen donnerstag den zwölften august endseq'

In [196]:
gloss_token_dev = []
text_token_dev = []
for i in range(len(annotation_gloss_dev)):
  gloss_token_dev.append(annotation_gloss_dev.iloc[i,2])
  text_token_dev.append(annotation_gloss_dev.iloc[i,3])

In [197]:
gloss_token_test = []
text_token_test = []
for i in range(len(annotation_gloss_test)):
  gloss_token_test.append(annotation_gloss_test.iloc[i,2])
  text_token_test.append(annotation_gloss_test.iloc[i,3])

# tokenizing whole glosses

In [198]:

def encode_sequences(tokenizer, length, lines):
	# integer encode sequences
	X = tokenizer.texts_to_sequences(lines)
	# pad sequences with 0 values
	X = pad_sequences(X, maxlen=length, padding='post')
	return X

In [199]:
X_train = encode_sequences(tokenizer,35,gloss_token)

In [200]:
X_dev = encode_sequences(tokenizer,35,gloss_token_dev)

In [201]:
X_test = encode_sequences(tokenizer,35,gloss_token_test)

In [134]:
X_dev.shape

(519, 35)

In [203]:
def encode_output(sequences, vocab_size):
  ylist = list()
  for sequence in sequences:
    encoded = to_categorical(sequence, num_classes=vocab_size)
    ylist.append(encoded)
  y = np.array(ylist)
  print("shape of sequences {}".format(np.shape(sequences)))
  y = y.reshape(sequences.shape[0], sequences.shape[1], vocab_size)
  return y


In [204]:
Y_output_token = encode_sequences(tokenizer,54,text_token)

In [137]:
Y_output = encode_output(Y_output_token,1202)

IndexError: ignored

In [205]:
Y_dev_token = encode_sequences(tokenizer,54,text_token_dev)

In [138]:
Y_dev = encode_output(Y_dev_token,1202)

shape of sequences (519, 54)


In [206]:
Y_test_token = encode_sequences(tokenizer,54,text_token_test)

In [143]:
Y_test = encode_output(Y_test_token,1202)

IndexError: ignored

# creating a function to return index corresponding to a word

In [207]:

def convert(lang, tensor):
  for t in tensor:
    if t!=0:
      print ("%d ----> %s" % (t, lang.index_word[t]))

In [208]:

print ("Input Language; index to word mapping")
convert(tokenizer, X_train[0])

Input Language; index to word mapping
199 ----> startseq
41 ----> jetzt
208 ----> wetter
40 ----> morgen
203 ----> donnerstag
209 ----> zwoelf
210 ----> februar
207 ----> endseq


In [209]:
print ("Target Language; index to word mapping")
convert( tokenizer, Y_output_token[0])

Target Language; index to word mapping
199 ----> startseq
39 ----> und
200 ----> nun
75 ----> die
201 ----> wettervorhersage
202 ----> für
40 ----> morgen
203 ----> donnerstag
204 ----> den
205 ----> zwölften
206 ----> august
207 ----> endseq


# creating a data batch

In [210]:
BUFFER_SIZE = len(X_train)
BATCH_SIZE = 128
steps_per_epoch = len(X_train)//BATCH_SIZE
embedding_dim = 256
units = 1024
vocab_inp_size = len(tokenizer.word_index)+1
vocab_tar_size = len(tokenizer.word_index)+1

dataset = tf.data.Dataset.from_tensor_slices((X_train, Y_output_token))
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
type(dataset)

tensorflow.python.data.ops.dataset_ops.BatchDataset

In [211]:
vocab_inp_size

3434

# creating a batch of 128 dataitemx with gloss length of 35  and text length of 54

In [212]:
example_input_batch, example_target_batch = next(iter(dataset))
example_input_batch.shape, example_target_batch.shape

(TensorShape([128, 35]), TensorShape([128, 54]))

# writhing encoder and decoder model

In [213]:

class Encoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, enc_units, batch_sz):
    super(Encoder, self).__init__()
    self.batch_sz = batch_sz
    self.enc_units = enc_units
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(self.enc_units,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform')

  def call(self, x, hidden):
    x = self.embedding(x)
    output, state = self.gru(x, initial_state = hidden)
    return output, state

  def initialize_hidden_state(self):
    return tf.zeros((self.batch_sz, self.enc_units))

In [214]:

encoder = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE)

# sample input
sample_hidden = encoder.initialize_hidden_state()
sample_output, sample_hidden = encoder(example_input_batch, sample_hidden)
print ('Encoder output shape: (batch size, sequence length, units) {}'.format(sample_output.shape))
print ('Encoder Hidden state shape: (batch size, units) {}'.format(sample_hidden.shape))

Encoder output shape: (batch size, sequence length, units) (128, 35, 1024)
Encoder Hidden state shape: (batch size, units) (128, 1024)


In [215]:
class BahdanauAttention(tf.keras.layers.Layer):
  def __init__(self, units):
    super(BahdanauAttention, self).__init__()
    self.W1 = tf.keras.layers.Dense(units)
    self.W2 = tf.keras.layers.Dense(units)
    self.V = tf.keras.layers.Dense(1)

  def call(self, query, values):
    # hidden shape == (batch_size, hidden size)
    # hidden_with_time_axis shape == (batch_size, 1, hidden size)
    # we are doing this to perform addition to calculate the score
    hidden_with_time_axis = tf.expand_dims(query, 1)

    # score shape == (batch_size, max_length, 1)
    # we get 1 at the last axis because we are applying score to self.V
    # the shape of the tensor before applying self.V is (batch_size, max_length, units)
    score = self.V(tf.nn.tanh(
        self.W1(values) + self.W2(hidden_with_time_axis)))

    # attention_weights shape == (batch_size, max_length, 1)
    attention_weights = tf.nn.softmax(score, axis=1)

    # context_vector shape after sum == (batch_size, hidden_size)
    context_vector = attention_weights * values
    context_vector = tf.reduce_sum(context_vector, axis=1)

    return context_vector, attention_weights

In [216]:
attention_layer = BahdanauAttention(10)
attention_result, attention_weights = attention_layer(sample_hidden, sample_output)

print("Attention result shape: (batch size, units) {}".format(attention_result.shape))
print("Attention weights shape: (batch_size, sequence_length, 1) {}".format(attention_weights.shape))

Attention result shape: (batch size, units) (128, 1024)
Attention weights shape: (batch_size, sequence_length, 1) (128, 35, 1)


In [217]:
class Decoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, dec_units, batch_sz):
    super(Decoder, self).__init__()
    self.batch_sz = batch_sz
    self.dec_units = dec_units
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(self.dec_units,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform')
    self.fc = tf.keras.layers.Dense(vocab_size)

    # used for attention
    self.attention = BahdanauAttention(self.dec_units)

  def call(self, x, hidden, enc_output):
    # enc_output shape == (batch_size, max_length, hidden_size)
    context_vector, attention_weights = self.attention(hidden, enc_output)

    # x shape after passing through embedding == (batch_size, 1, embedding_dim)
    x = self.embedding(x)

    # x shape after concatenation == (batch_size, 1, embedding_dim + hidden_size)
    x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)

    # passing the concatenated vector to the GRU
    output, state = self.gru(x)

    # output shape == (batch_size * 1, hidden_size)
    output = tf.reshape(output, (-1, output.shape[2]))

    # output shape == (batch_size, vocab)
    x = self.fc(output)

    return x, state, attention_weights

In [218]:
decoder = Decoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE)

sample_decoder_output, _, _ = decoder(tf.random.uniform((BATCH_SIZE, 1)),
                                      sample_hidden, sample_output)

print ('Decoder output shape: (batch_size, vocab size) {}'.format(sample_decoder_output.shape))

Decoder output shape: (batch_size, vocab size) (128, 3434)


In [219]:
optimizer = tf.keras.optimizers.Adam()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

def loss_function(real, pred):
  mask = tf.math.logical_not(tf.math.equal(real, 0))
  loss_ = loss_object(real, pred)

  mask = tf.cast(mask, dtype=loss_.dtype)
  loss_ *= mask

  return tf.reduce_mean(loss_)

In [220]:

checkpoint_dir = 'training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(optimizer=optimizer,
                                 encoder=encoder,
                                 decoder=decoder)

# training

In [221]:
@tf.function
def train_step(inp, targ, enc_hidden):
  loss = 0

  with tf.GradientTape() as tape:
    enc_output, enc_hidden = encoder(inp, enc_hidden)

    dec_hidden = enc_hidden

    dec_input = tf.expand_dims([tokenizer.word_index['startseq']] * BATCH_SIZE, 1)

    # Teacher forcing - feeding the target as the next input
    for t in range(1, targ.shape[1]):
      # passing enc_output to the decoder
      predictions, dec_hidden, _ = decoder(dec_input, dec_hidden, enc_output)

      loss += loss_function(targ[:, t], predictions)

      # using teacher forcing
      dec_input = tf.expand_dims(targ[:, t], 1)

  batch_loss = (loss / int(targ.shape[1]))

  variables = encoder.trainable_variables + decoder.trainable_variables

  gradients = tape.gradient(loss, variables)

  optimizer.apply_gradients(zip(gradients, variables))

  return batch_loss

In [222]:
EPOCHS = 20
with tf.device('/gpu:0'):
  for epoch in range(EPOCHS):
    start = time.time()

    enc_hidden = encoder.initialize_hidden_state()
    total_loss = 0

    for (batch, (inp, targ)) in enumerate(dataset.take(steps_per_epoch)):
      batch_loss = train_step(inp, targ, enc_hidden)
      total_loss += batch_loss
      if batch % 100 == 0:
        print('Epoch {} Batch {} loss {}'.format(epoch + 1,batch, batch_loss.numpy()))
    
        
    # saving (checkpoint) the model every 2 epochs
    if (epoch + 1) % 2 == 0:
      checkpoint.save(file_prefix = checkpoint_prefix)

    print('Epoch {} Loss {:.4f}'.format(epoch + 1,
                                        total_loss / steps_per_epoch))
    print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

Epoch 1 Batch 0 loss 2.2815701961517334
Epoch 1 Loss 1.6391
Time taken for 1 epoch 92.99105310440063 sec

Epoch 2 Batch 0 loss 1.46639084815979
Epoch 2 Loss 1.3358
Time taken for 1 epoch 39.31737971305847 sec

Epoch 3 Batch 0 loss 1.156550407409668
Epoch 3 Loss 1.1297
Time taken for 1 epoch 39.71906805038452 sec

Epoch 4 Batch 0 loss 1.018256425857544
Epoch 4 Loss 1.0250
Time taken for 1 epoch 40.837549448013306 sec

Epoch 5 Batch 0 loss 0.9535468816757202
Epoch 5 Loss 0.9591
Time taken for 1 epoch 40.81982159614563 sec

Epoch 6 Batch 0 loss 0.8882601857185364
Epoch 6 Loss 0.9030
Time taken for 1 epoch 41.55751061439514 sec

Epoch 7 Batch 0 loss 0.839790940284729
Epoch 7 Loss 0.8584
Time taken for 1 epoch 41.348894357681274 sec

Epoch 8 Batch 0 loss 0.8055030703544617
Epoch 8 Loss 0.8193
Time taken for 1 epoch 41.95289731025696 sec

Epoch 9 Batch 0 loss 0.7672399878501892
Epoch 9 Loss 0.7811
Time taken for 1 epoch 41.68753242492676 sec

Epoch 10 Batch 0 loss 0.7310289740562439
Epoch 10

# translate

In [223]:
max_target_length = 54
max_source_length = 35
def evaluate(sentence):
 # attention_plot = np.zeros((max_target_length, max_source_length))


  #sentence = preprocess_sentence(sentence)
  #print(sentence)
  #print(source_sentence_tokenizer.word_index)

  inputs = [tokenizer.word_index[i] for i in sentence.split(' ')]
  inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs],
                                                         maxlen=35,
                                                         padding='post')
  inputs = tf.convert_to_tensor(inputs)

  result = ''

  hidden = [tf.zeros((1, units))]
  enc_out, enc_hidden = encoder(inputs, hidden)

  dec_hidden = enc_hidden
  dec_input = tf.expand_dims([tokenizer.word_index['startseq']], 0)

  for t in range(max_target_length):
    predictions, dec_hidden, attention_weights = decoder(dec_input,
                                                         dec_hidden,
                                                         enc_out)

    # storing the attention weights to plot later on
    attention_weights = tf.reshape(attention_weights, (-1, ))
   # attention_plot[t] = attention_weights.numpy()

    predicted_id = tf.argmax(predictions[0]).numpy()

    result += tokenizer.index_word[predicted_id] + ' '

    if tokenizer.index_word[predicted_id] == 'endseq':
      #return result, sentence, attention_plot
      return result,sentence

    # the predicted ID is fed back into the model
    dec_input = tf.expand_dims([predicted_id], 0)

  #return result, sentence, attention_plot
  return result, sentence

In [224]:
def translate(sentence):
  #result, sentence, attention_plot = evaluate(sentence)
  result,sentence = evaluate(sentence)
  
  print('Input: %s' % (sentence))
  print('Predicted translation: {}'.format(result))

  #attention_plot = attention_plot[:len(result.split(' ')), :len(sentence.split(' '))]

In [225]:
for i in range(10):
  translate(annotation_gloss_test.iloc[i,2])
  print("******")
  print(annotation_gloss_test.iloc[i,3])

Input: startseq regen schnee region verschwinden nord regen koennen region stern koennen sehen endseq
Predicted translation: vor allem in der nacht regnet es hier und da fällt regen der regen fällt regen der regen fällt regen der regen fällt regen der regen fällt regen der regen fällt regen der regen fällt regen der regen fällt regen der regen fällt regen der regen fällt regen der regen fällt regen der regen 
******
startseq regen und schnee lassen an den alpen in der nacht nach im norden und nordosten fallen hier und da schauer sonst ist das klar endseq
Input: startseq donnerstag nordwest regen region sonne wolke wechselhaft dann freitag aehnlich wetter endseq
Predicted translation: am donnerstag regnet es im norden und westen und westen und westen und westen und westen und westen und westen und westen und westen und westen und westen und westen und westen und westen und westen und westen und westen und westen und westen und westen und westen und westen und westen und westen 
******
s

KeyError: ignored